# <a href="http://blog.yhat.com/posts/the-beer-bandit.html">Python Multi-armed Bandits (and Beer!)</a>
The term multi-armed bandit comes from a theoretical problem where instead of one slot machine lever, you have a number of them, say three like in the image above. If you know the odds and payouts, it's trivial to determine the lever with the highest expected value. But imagine you didn't know the odds (like not knowing which model preforms best). A bandit algorithm attempts to solve this problem and maximize profit by systematically testing different levers and remembering the rewards.

So what does this have to do with beer recommenders? What the analogy of a multi-armed slot machine captures well is it costs to test your hypotheses. A slot machine takes a coin to play. A website costs you a visit. Testing gains you knowledge, but will sometimes necessitate supplying your customers with an inferior product. In response, bandit algorithms are automated to move away from low scoring options and inherently prefer those that preform better.

Of course you can do this with an A/B test. And if you're only going to run one or two test over a short period of time which you track frequently, an A/B test will often perform just as well as a bandit algorithm. But as the number and length of hypotheses and tests increase, the advantage of an automatic feedback loop allows the designer to take a more hands off approach.

## Epsilon-Greedy
Despite its simplicity, the epsilon-greedy algorithm does a good job of encapsulating the spirit of bandit algorithms. Every time the algorithm has to choose an option (also referred to as an arm), it first considers two possibilities; **explore** or **exploit**. Explore corresponds to testing, and if epsilon-greedy takes this path it simply chooses an arm at random. The exploit path on the other hand leads only to the arm with the best performance so far, the safe bet.

The "best arm" is considered the option (in our case a recommendation model) with the highest probability of reward. I'll define my feedback system later, but for now think of reward as akin to *clicks/views*. Below is a probabilistic diagram which expresses this behavior. Notice that once the system lands on the "testing" node, it's synonyms with generalized A/B testing.
<img style="width:50%" src="data/epsilongreedy.png">

Epsilon-greedy will choose to test with a frequency of epsilon (hence the name). An epsilon value of  will cause it to always test, while a value of  will cause it to always exploit the best preforming arm.

In [2]:
__filename__ = "bandits.py"

import numpy as np

class EsilonGreedy(object):
    def __init__(self, n_arms, epsilon_decay=50):
        self.counts = [0] * n # example: number of views
        self.values = [0.] * n # example: number of clicks / views
        self.decay = epsilon_decay
        self.n = n_arms
        
        
    def choose_arm(self):
        """Choose an arm for testing"""
        epsilon = self.get_epsilon()
        if np.random.random() > epsilon:
            # Exploit (Use best arm)
            return np.argmax(self.values)
        else:
            # Explore (test all arms)
            return np.random.randint(self.n)
        
    def update(self, arm, reward):
        """
        Update an arm with some reward value
        Example: click = 1; no click = 0
        """
        self.counts[arm] = self.counts[arm] + 1
        n = self.counts[arm]
        value = self.values[arm]
        # Running product
        new_value = ((n - 1) / float(n)) * value + (1 / float(n)) * reward
        self.values[arm] = new_value
        
    def get_epsilon(self):
        """Produce epsilon"""
        total = np.sum(arm_counts)
        return float(self.decay) / (total + float(self.decay))

While epsilon is normally a fixed value, I've implemented a method called get_epsilon() which produces a variable value based on the total amount of feedback. Let's take a look at what the embedded equation looks like using ggplot.

In [3]:
from ggplot import *
%matplotlib inline

ggplot(pd.DataFrame({'x':[0,500]}),aes(x='x')) + \
    stat_function(fun = lambda x: 50. / (x + 50.)) + \
    ggtitle("Epsilon decay over time") + \
    xlab("amount of feedback") + ylab("epsilon (probability of testing)")


ModuleNotFoundError: No module named 'ggplot'

## Three different beer recommenders

In [23]:
import pandas as pd
import numpy as np

df = pd.read_csv("data/beer_reviews.csv")
# Limit to the top 250 beers
top_n = df.beer_name.value_counts().index[:250]
df = df[df.beer_name.isin(top_n)]

# Create a matric of beer v.s. reviews; values are overall review score
df_wide = pd.pivot_table(df, values=["review_overall"],
        index=["beer_name", "review_profilename"],
        aggfunc=np.mean).unstack()

df_wide = df_wide.fillna(0)
df_wide


review_overall                                   \
review_profilename            0runkp0s 1099.0 11osixBrew 12NattiBottles   
beer_name                                                                 
60 Minute IPA                      0.0    0.0        4.0            0.0   
Blue Moon Belgian White            0.0    0.0        0.0            0.0   
Bud Light                          5.0    4.0        0.0            0.0   
Coors Light                        0.0    3.0        0.0            0.0   
Dale's Pale Ale                    0.0    0.0        0.0            0.0   
Fat Tire Amber Ale                 0.0    0.0        0.0            0.0   
Guinness Draught                   0.0    0.0        0.0            0.0   
Michelob Ultra                     0.0    4.0        0.0            0.0   
Natural Light                      0.0    1.5        0.0            0.0   
Sierra Nevada Pale Ale             0.0    0.0        0.0            3.5   

                                                                          \
review_profilename      1759Girl 1759dallas 17Guinness59 18alpha 1Adam12   
beer_name                                                                  
60 Minute IPA                0.0        0.0          0.0     0.0     0.0   
Blue Moon Belgian White      0.0        0.0          0.0     0.0     0.0   
Bud Light                    0.0        0.0          0.0     0.0     0.0   
Coors Light                  0.0        0.0          0.0     0.0     0.0   
Dale's Pale Ale              0.0        0.0          0.0     0.0     0.0   
Fat Tire Amber Ale           0.0        0.0          0.0     0.0     0.0   
Guinness Draught             5.0        4.5          3.5     0.0     0.0   
Michelob Ultra               0.0        0.0          0.0     0.0     0.0   
Natural Light                0.0        0.0          0.0     0.0     0.0   
Sierra Nevada Pale Ale       4.5        0.0          0.0     3.5     3.5   

                                     ...                                      \
review_profilename      1fastz28     ...     zoso1967 zoso493 zpeid zplug123   
beer_name                            ...                                       
60 Minute IPA                4.0     ...          4.5     4.0   0.0      4.0   
Blue Moon Belgian White      3.0     ...          0.0     0.0   0.0      0.0   
Bud Light                    0.0     ...          0.0     0.0   0.0      0.0   
Coors Light                  0.0     ...          0.0     0.0   0.0      0.0   
Dale's Pale Ale              0.0     ...          4.5     0.0   0.0      4.0   
Fat Tire Amber Ale           4.5     ...          0.0     3.5   0.0      0.0   
Guinness Draught             4.5     ...          0.0     0.0   0.0      0.0   
Michelob Ultra               0.0     ...          0.0     0.0   0.0      0.0   
Natural Light                0.0     ...          0.0     0.0   0.0      0.0   
Sierra Nevada Pale Ale       4.5     ...          0.0     4.5   3.5      0.0   

                                                                     \
review_profilename      zrab11 zseeanz ztaylor1 zuffenhausen zuggy9   
beer_name                                                             
60 Minute IPA              4.5     4.5      0.0          4.5    0.0   
Blue Moon Belgian White    0.0     0.0      2.0          0.0    0.0   
Bud Light                  0.0     0.0      0.0          0.0    0.0   
Coors Light                0.0     0.0      0.0          0.0    1.0   
Dale's Pale Ale            0.0     0.0      0.0          0.0    0.0   
Fat Tire Amber Ale         0.0     0.0      0.0          0.0    4.5   
Guinness Draught           4.0     0.0      0.0          0.0    4.0   
Michelob Ultra             0.0     0.0      0.0          0.0    0.0   
Natural Light              0.0     0.0      0.0          0.0    0.0   
Sierra Nevada Pale Ale     4.5     4.5      4.0          0.0    4.0   

                                     
review_profilename      zymurgy4all  
beer_name         

The main emphasis of our recommendation system is the distance between beers. By distance, I mean the distance between the two rows representing each beer. This eventually boils down to comparing two vectors and their "similarity". In simple terms, beers that are closer by some metric should be more related and therefore a better recommendation.

But in multidimensional space there are a lot of ways to measure the distance and similarity between two points. Here I pick three measures: Euclidean distance, distance correlation, and cosine similarity. From there, I create three different distance matrices and define a helper function.

In [18]:
from sklearn.metrics.pairwise import (
    euclidean_distances, cosine_similarity, pairwise_distances
)

# Calculate euclidean distance between each beer
eucl_dists = euclidean_distances(df_wide)
eucl_dists = pd.DataFrame(eucl_dists, columns=df_wide.index)
eucl_dists.index = eucl_dists.columns

# Calculate cosine similarity
cos_dists = cosine_similarity(df_wide)
cos_dists = pd.DataFrame(cos_dists, columns=df_wide.index)
cos_dists.index = cos_dists.columns

# Calculate distance correlation
corr_dists = pairwise_distances(df_wide,metric='correlation')
corr_dists = pd.DataFrame(corr_dists, columns=df_wide.index)
corr_dists.index = corr_dists.columns

# Use distance matrix to determine similarity
def get_sims(products, dists, larger_is_closer=False):
    """Return similarity matrix"""
    p = dists[products].apply(lambda row: np.sum(row), axis=1)
    p = p.order(ascending=larger_is_closer)
    return p.index[p.index.isin(products)==False]

In [27]:
products = ["Sierra Nevada Pale Ale", "Bud Light", "Coors Light"]

eucl_prods = get_sims(products,eucl_dists)[:10]
cos_prods = get_sims(products,cos_dists,larger_is_closer=True)[:10]
corr_prods = get_sims(products,corr_dists)[:10]

print("Products similar to:", ', '.join(products))
pd.DataFrame({'Euclidean Distance': eucl_prods,
        "Cosine Similarity":cos_prods,
        "Distance Correlation":corr_prods})

AttributeError: 'Series' object has no attribute 'order'